# import

In [2]:
import sys
sys.path.append('.')
import os,sys
import h5py
import numpy as np
from scipy import signal
import time
import skimage
import skimage.io, skimage.transform
from skimage.transform import resize
from skimage.util import view_as_windows
import scipy.misc
import scipy.io as sio
from utils import *
from skimage import img_as_uint
import matplotlib.pyplot as plt
# import pandas
import glob 
import datetime
import math
from skimage.color import rgb2ycbcr 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import pywt
# import cv2
import re
from PIL import Image
import os
import bisect
import pickle
from PIL import Image

from sys import argv
import scipy.io as sio
from time import time
import numpy as np
import skimage.morphology as ski
from feat_spam.spam_np import getParams as getSpamParams
# from feat_spam.spam_np import getSpam
from utility.utilityImage import img2grayf

from utility.utilityImage import imread2f
from feat_spam.residue import getFiltersResidue
import feat_spam.mapping as spam_m
from scipy.signal import correlate2d
from skimage import io,data,color
import matplotlib.pyplot as plt

# define the division of training and testing

In [3]:
def train_test_split1(images, labels):
    """ 将图片和标签划分为训练集和测试集
    
    :params images: 图像。ndarray。
    :params labels: 标签。ndaary。
    :params train_ratio: 训练集比例
    :params test_ratio: 测试集比例
    :return: 
    """
    train_ratio = 0.75
    RANDOM_SEED = 100
    np.random.seed(RANDOM_SEED)
    image_num = len(images)
    print('image_num =', len(images))
    rand_idx = np.arange(image_num)
    np.random.shuffle(rand_idx)
    train_idx = rand_idx[:int(train_ratio * image_num)]
#     print('train_idx=',train_idx)
    print(train_idx.dtype)
    train_idx = train_idx.astype(np.int32)
    print(train_idx.dtype)
    test_idx = rand_idx[int(train_ratio * image_num):]
    train_images = images[train_idx]
    train_labels = labels[train_idx]
    
    test_images = images[test_idx]
    test_labels = labels[test_idx]
    return train_images, train_labels,  test_images, test_labels


# extract resampling features

In [8]:
def reample_feature_computer(imgs):
    print('np.shape(imgs)[0]=',np.shape(imgs)[0])
    for q in range(0,np.shape(imgs)[0]):
        sys.stdout.write('\r' + str(q))
        sys.stdout.flush()
        im=imgs[q]
        img_feat = np.ndarray([np.shape(imgs)[0], 64,240], dtype=np.float32)
        # extract square patches with stride (step) 8
        patchsize=32
        # reshape to a list of patches
        rgb_patches = view_as_windows(im,(32,32,3),32)
        rgb_patches = np.squeeze(rgb_patches)
        listofpatches = np.reshape(rgb_patches,(64,32,32,3))
        #listofpatches = orig_object.reshape((1, patchsize, patchsize, 3))
        #print("patches array reshaped to list of patches with shape "+str(listofpatches.shape))

        # Radon projection parameters
        circle_inscribed = False
        numAngles = 10
        theta = np.linspace(0,180,numAngles,endpoint=False)

        def radon_projections_compiled_cuda(patches, thetas, circle_inscribed):
            sys.path.append(os.path.join(thispath,'build')) # for importing pysinogram.so
            from pysinogram import BatchRadonTransform
            return np.array(BatchRadonTransform(list(patches), list(thetas), circle_inscribed))

        def radon_projections_skimage_python(patches, thetas, circle_inscribed):
            # sqrt(abs(  2D discrete 3x3 laplacian filter  ))
            kernel = np.array([[0, 1, 0], [1, -4, 1], [0, 1, 0]])
            laplacefilter = lambda xx: np.sqrt(np.fabs(signal.convolve2d(xx, kernel, mode='same', boundary='symm')))
            # do laplacian filter on each channel independently, then average across channels
            rgbfilter = lambda xx: np.mean([laplacefilter(xx[:,:,chan]) for chan in range(xx.shape[2])], axis=0)
            # transpose the sinogram output, to be consistent with CUDA implementation above
            myradon = lambda xx: skimage.transform.radon(rgbfilter(xx), theta=theta, circle=circle_inscribed).transpose()
            # iterate processing over all patches
            return np.stack([myradon(patches[ii,...]) for ii in range(patches.shape[0])], axis=0)

        # run tests
        if False:
            # compare against compiled implementation
            # requires compiling using cuda-radon-transform repository, available on Bitbucket
            t0 = time.time()
            check11 =  radon_projections_compiled_cuda(listofpatches, theta, circle_inscribed)
            t1 = time.time()
            check22 = radon_projections_skimage_python(listofpatches, theta, circle_inscribed)
            t2 = time.time()
            print("Radon projections time, compiled CUDA:  "+str(t1-t0)+" seconds")
            print("Radon projections time, python skimage: "+str(t2-t1)+" seconds")
            describe("check11", check11)
            describe("check22", check22)

            import cv2
            for ii in range(check11.shape[0]):
                checkdiff = np.fabs(check11[ii,:,:] - check22[ii,:,:])
                describe("checkdiff", checkdiff)
                zp = np.zeros((4,check11.shape[2]))
                concat = np.concatenate((check11[ii,:,:], zp, check22[ii,:,:], zp, checkdiff), axis=0)
                #cv2.imshow("npresult", uint8norm(concat))
                #cv2.waitKey(0)
        else:
            # run only one of the implementations
            radonfunc = radon_projections_skimage_python
#             beftime = time.time()
            npresult = radonfunc(listofpatches, theta, circle_inscribed)
            #print("sinogram calculation took "+str(time.time()-beftime)+" seconds")
            #describe("python sinogram", npresult)
            assert len(npresult.shape) == 3, str(npresult.shape)

            # also do FFT + normalization as final stage of feature extraction
            # subtract 1 from normed which is the mean

            absproc = lambda xx: np.expand_dims(np.absolute(xx), axis=-1)
#             beftime = time.time()
            _, fftnormed, _, fftavg = fftscores(npresult)

            npresult = absproc(fftnormed) - 1.
            #npresult = np.concatenate([absproc(fftnormed) - 1., absproc(fftavg)], axis=1)
            npresult=np.transpose(npresult,(3,0,1,2))

            npresult=np.reshape(npresult,(64,240))
            print ("feature extrating for image # "+ str(q+1)+", with shape-->"+str(np.shape(npresult)))
            #print("FFT calculations took "+str(time.time()-beftime)+" seconds")
            #describe("npresult", npresult)
#             hdf5_file["feat"][q, ...] = npresult[None]
#             img_feat.append(npresult)
            img_feat[q, ...] = npresult[None]
#     img_feat = np.array(img_feat, dtype=np.float32)
    print('img_feat_shape=',img_feat.shape)
    return img_feat

# extract the features of the co-occurrence matrix

In [4]:
defaultSpamParam = getSpamParams(ordResid=3, symTranspose=True, q=2, T=1, ordCooc=4, mapper='SignSym', strides=8)
smallSpamParam   = getSpamParams(ordResid=3, symTranspose=True, q=2, T=1, ordCooc=4, mapper='SignSym', strides=4)
def SB_main(imgfilename):
    img = imgfilename
    imgsize = img.shape
    print('imgsize:',imgsize)
    if imgsize[0]*imgsize[1]>20000:
         mapp = SBgu(img, paramSpam = defaultSpamParam, ksize = 128, flagDropSaturated=2, paddingModality=0)
    else:
         mapp = SBgu(img, paramSpam = smallSpamParam, ksize =  64, flagDropSaturated=2, paddingModality=0)
    
    return mapp

def SBgu(img, paramSpam = defaultSpamParam, ksize = 128, flagDropSaturated = 2, satutationProb = 0.85, extFeat = range(25), seed = 0, maxIter = 100, replicates = 30, outliersNlogl = 42, paddingModality = 0, flagShow = False):
    if flagDropSaturated > 0:
        if img.dtype== np.uint8:
            weights = np.logical_not(getSatMap(img, 6, 252)).astype(np.float32)
        else:
            weights = np.logical_not(getSatMap(img, 6.0 / 256, 252.0 / 256)).astype(np.float32)
        weights = ski.binary_erosion(weights, np.ones((2*paramSpam['radius'][0]+1, 2*paramSpam['radius'][1]+1), dtype = np.bool))
    else:
        weights = list()
    
#     img = img2grayf(img)
    img = color.rgb2hsv(img)
    img1 = img[:,:,0]
    img2 = img[:,:,1]
    img3 = img[:,:,2]
#     print('img_size:',img.shape)
    spam1 = getSpam(img1, paramSpam, ksize, weights = weights, paddingModality = paddingModality)
    spam2 = getSpam(img2, paramSpam, ksize, weights = weights, paddingModality = paddingModality)
    spam3 = getSpam(img3, paramSpam, ksize, weights = weights, paddingModality = paddingModality)
    return spam1,spam2,spam3

def getSpam(X, params, ksize, weights = list(), paddingModality = 0):

    strides = params['strides']
    if isinstance(ksize, (list,tuple)):
        ksize = ksize[0:2]
    else:
        ksize = [ksize, ksize]

    ksize[0] = int(ksize[0] / strides[0])
    ksize[1] = int(ksize[1] / strides[1])

    spam = computeSpam(X, params, weights = weights, normalize=True)
    
    return spam


def getSatMap(img, th_Black, th_White):
    if len(img.shape) > 2:
        sat_mask0 = ski.binary_opening(img[:,:,0] < th_Black, ski.disk(3))
        sat_mask1 = ski.binary_opening(img[:,:,0] > th_White, ski.disk(2))
        for index in range(1, img.shape[2]):
            sat_mask0 = np.logical_or(sat_mask0, ski.binary_opening(img[:, :, index] < th_Black, ski.disk(3)))
            sat_mask1 = np.logical_or(sat_mask1, ski.binary_opening(img[:, :, index] > th_White, ski.disk(2)))
    else:
        sat_mask0 = ski.binary_opening(img<th_Black,ski.disk(3))
        sat_mask1 = ski.binary_opening(img>th_White,ski.disk(2))

    return np.logical_or(sat_mask0,sat_mask1)

def computeSpam(X, params, weights = list(),  normalize = True):
    
#     print('X:',X.shape)
    ## Residue
    WresH = params['Wres']
    WresV = WresH.transpose([1,0,2,3])
    resH = correlate2d(X.squeeze(), WresH.squeeze(), mode='valid')
    resV = correlate2d(X.squeeze(), WresV.squeeze(), mode='valid')
#     print('resH:',resH.shape)
    ## Quantization & Truncation
    values = params['values']
    if params['uniformQuant']:
        n = values.size
        T = (n - 1) / 2
        q = (values[1] - values[0])
        resHq = (np.clip(my_round(resH / q) + T, 0, n - 1)).astype(np.int64)
        resVq = (np.clip(my_round(resV / q) + T, 0, n - 1)).astype(np.int64)
    else:
        resHq = quantizerScalarEncoder(resH, values)
        resVq = quantizerScalarEncoder(resV, values)
    
#     print('resHq:',resHq.shape)
    ## Coocorance
    ordCooc = params['ordCooc']
    n = (params['values']).size
    dim = int(ordCooc + 1 - (ordCooc % 2))
    indexL = int((dim - 1) / 2)
#     indexL =0
    shapeR = np.asarray(resHq.shape[:2]) - dim +1 
#     shapeR = np.asarray(resHq.shape)
#     print('shapeR:',shapeR)
    resHh = np.zeros(shapeR, dtype = np.int)
    resVh = np.zeros(shapeR, dtype = np.int)
    resHv = np.zeros(shapeR, dtype = np.int)
    resVv = np.zeros(shapeR, dtype = np.int)

    for indexP in range(ordCooc):
        nn = (n ** indexP)
        
#         print('n---:', n)
#         print('indexp==:', indexP)
#         print('===nn:',nn)
#         print(indexL,(shapeR[0] + indexL))
#         print(indexP,(shapeR[1] + indexP))
        
        resHh += resHq[indexL:(shapeR[0] + indexL), indexP:(shapeR[1] + indexP)] * nn
        resVh += resVq[indexL:(shapeR[0] + indexL), indexP:(shapeR[1] + indexP)] * nn
        resHv += resHq[indexP:(shapeR[0] + indexP), indexL:(shapeR[1] + indexL)] * nn
        resVv += resVq[indexP:(shapeR[0] + indexP), indexL:(shapeR[1] + indexL)] * nn
#     print('resHh:',resHh.shape)
#     print('resVh:',resVh.shape)
#     print('resHv:',resHv.shape)
#     print('resVv:',resVv.shape)
#     print('resHh:',resHh+resVh+resHv+resVv)
    ## Mappeing
    mapper = params['mapper']
    if len(mapper) > 0:
        resHh = mapper['table'][resHh].squeeze()
        resVh = mapper['table'][resVh].squeeze()
        resHv = mapper['table'][resHv].squeeze()
        resVv = mapper['table'][resVv].squeeze()
#     print('resHh:',resHh.shape)
#     print('resVh:',resVh.shape)
#     print('resHv:',resHv.shape)
#     print('resVv:',resVv.shape)
    
    
    co_occurrence = resHh+resVh+resHv+resVv
#     print('co_occurrence:',co_occurrence.shape)
    return co_occurrence

In [5]:
my_round = lambda x: np.sign(x) * np.floor(np.abs(x)+0.5)
def reample_feature_computer1(imgs):
    print('np.shape(imgs)[0]=',np.shape(imgs)[0])
    for q in range(0,np.shape(imgs)[0]):
        sys.stdout.write('\r' + str(q))
        sys.stdout.flush()
        im=imgs[q]
        img_feat = np.ndarray([np.shape(imgs)[0], 248,248,3], dtype=np.float32)
        my_round = lambda x: np.sign(x) * np.floor(np.abs(x)+0.5)
        
        mapp1,mapp2,mapp3 = SB_main(im)

        map = np.stack((mapp1,mapp2,mapp3),axis =2)
        img_feat[q, ...] = map[None]
    print('img_feat_shape=',img_feat.shape)
    return img_feat

# load dataset

## IFC

In [10]:
IFS_IMG_DIR_manipulation = '/home/shizenan/Datasets/IFS-TC-phase2-shi/ima'
IFS_IMG_MASK_DIR_manipulation = '/home/shizenan/Datasets/IFS-TC-phase2-shi/img_mask'

img_num_toal = len(glob.glob(IFS_IMG_DIR_manipulation+'/*'))
gt_num_toal = len(glob.glob(IFS_IMG_MASK_DIR_manipulation+'/*'))

print('img_num_toal = ',img_num_toal)#一共1064幅
print('gt_num_toal = ',gt_num_toal)

Cov_imgS = []
Cov_labelS = []

for i, img in enumerate(glob.glob(IFS_IMG_DIR_manipulation+'/*')):

    sys.stdout.write('\ri: %d\t%s'%(i+1, os.path.basename(img)))
    sys.stdout.flush()
    
    image = plt.imread(img)
    print()
    print('image.shape[2]=',image.shape[2])
    if image.shape[2]==4:
        image = image[:,:,0:3]
        image = np.array(image)
    print('image.shape[2]=',image.shape[2])
    print()
    
    image_mani=scipy.misc.imresize(image,(256,256,3),interp='bilinear',mode=None)
    plt.imsave(fname='../data_256/IFS/img/'+os.path.basename(img),arr=image_mani)
    mask_mani = plt.imread('/home/shizenan/Datasets/IFS-TC-phase2-shi/img_mask/'+os.path.basename(img))
    if len(mask_mani.shape)==3:
        mask_mani = mask_mani[:,:,0]
    print('mask_mani',mask_mani.shape)
    if len(mask_mani.shape)==4:
        mask_mani = mask_mani[:,:,0]
    print('mask_mani',mask_mani.shape)
    
    
    mask_mani = scipy.misc.imresize(mask_mani,(256,256),interp='bilinear',mode=None)
    class_labels_tensor = np.float32(mask_mani==0)
    plt.imsave(fname='../data_256/IFS/GT/'+os.path.basename(img),arr=class_labels_tensor, cmap=plt.cm.gray)
    
    Cov_imgS.append(image_mani)
    Cov_labelS.append(class_labels_tensor)


Cov_imgS = np.array(Cov_imgS,dtype = np.float32)
Cov_labelS = np.array(Cov_labelS,dtype = np.float32)

Cov_train_img,  Cov_train_label, Cov_test_img, Cov_test_label= train_test_split1(Cov_imgS, Cov_labelS)
Cov_train_img_num = len(Cov_train_img)
Cov_train_label_num = len(Cov_train_label)
print('Cov_train_img.shape',Cov_train_img.shape)
print('Cov_train_label.shape',Cov_train_label.shape)

Cov_test_img_num = len(Cov_test_img)
Cov_test_label_num = len(Cov_test_label)
print('Cov_test_img.shape',Cov_test_img.shape)
print('Cov_test_label.shape',Cov_test_label.shape)

print(len(Cov_train_label)+len(Cov_test_label))    
print()

np.save('../dataset_npy/IFS/IFS_train_img.npy', Cov_train_img)
np.save('../dataset_npy/IFS/IFS_train_label.npy', Cov_train_label)
np.save('../dataset_npy/IFS/IFS_test_img.npy', Cov_test_img)
np.save('../dataset_npy/IFS/IFS_test_label.npy', Cov_test_label)



img_num_toal =  441
gt_num_toal =  441
i: 1	279.png
image.shape[2]= 4
image.shape[2]= 3



/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


mask_mani (1536, 2048)
mask_mani (1536, 2048)
i: 2	103.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 3	98.png

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.



image.shape[2]= 4
image.shape[2]= 3

mask_mani (1536, 2048)
mask_mani (1536, 2048)
i: 4	73.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 5	46.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (765, 1024)
mask_mani (765, 1024)
i: 6	294.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 7	69.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 8	396.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (1200, 1600)
mask_mani (1200, 1600)
i: 9	78.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 10	429.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (691, 922)
mask_mani (691, 922)
i: 11	80.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (765, 1024)
mask_mani (765, 1024)
i: 12	265.png
image.shape[2]= 4
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 13	336.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (806, 1075)

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 90	109.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 91	232.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 92	177.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 93	221.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 94	195.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 95	357.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 96	305.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (765, 1024)
mask_mani (765, 1024)
i: 97	426.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (1000, 1504)
mask_mani (1000, 1504)
i: 98	101.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 99	285.png
image.shape[2]= 4
image.shape[2]= 3

mask_mani (1536, 2048)
mask_mani (1536, 2048)
i:

i: 175	189.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 176	222.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 177	277.png
image.shape[2]= 4
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 178	126.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 179	91.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 180	334.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (771, 1024)
mask_mani (771, 1024)
i: 181	81.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 182	162.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 183	377.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (765, 1024)
mask_mani (765, 1024)
i: 184	29.png
image.shape[2]= 4
image.shape[2]= 3

mask_mani (600, 800)
mask_mani (600, 800)
i: 185	261.png
image.shape[2]= 3
image.shape[

mask_mani (2304, 3072)
mask_mani (2304, 3072)
i: 261	205.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 262	320.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (1944, 2592)
mask_mani (1944, 2592)
i: 263	37.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (2304, 3072)
mask_mani (2304, 3072)
i: 264	190.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 265	44.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 266	398.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 267	352.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (806, 1075)
mask_mani (806, 1075)
i: 268	240.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 269	169.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 270	361.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (3240, 4320)
mask_mani (32

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 346	119.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 347	373.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (1067, 1600)
mask_mani (1067, 1600)
i: 348	39.png
image.shape[2]= 4
image.shape[2]= 3

mask_mani (1536, 2048)
mask_mani (1536, 2048)
i: 349	433.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (1944, 2592)
mask_mani (1944, 2592)
i: 350	313.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (765, 1024)
mask_mani (765, 1024)
i: 351	435.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (806, 1075)
mask_mani (806, 1075)
i: 352	33.png
image.shape[2]= 4
image.shape[2]= 3

mask_mani (2758, 4150)
mask_mani (2758, 4150)
i: 353	123.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 354	1.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (1536, 2048)
mask_mani (1536, 2048)
i: 355	260.png
image.shape[2]= 4
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (7

i: 431	120.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 432	18.png
image.shape[2]= 4
image.shape[2]= 3

mask_mani (1200, 1800)
mask_mani (1200, 1800)
i: 433	286.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 434	258.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (1936, 2592)
mask_mani (1936, 2592)
i: 435	238.png
image.shape[2]= 4
image.shape[2]= 3

mask_mani (1536, 2048)
mask_mani (1536, 2048)
i: 436	62.jpg
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 437	124.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 438	50.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 439	268.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (771, 1024)
mask_mani (771, 1024)
i: 440	15.png
image.shape[2]= 4
image.shape[2]= 3

mask_mani (933, 1600)
mask_mani (933, 1600)
i: 441	110.png
image.shape[2]= 3
image

In [13]:
IFS_IMG_DIR_manipulation = '/home/shizenan/Datasets/IFS-TC-phase2-shi/ima'
IFS_IMG_MASK_DIR_manipulation = '/home/shizenan/Datasets/IFS-TC-phase2-shi/img_mask'

img_num_toal = len(glob.glob(IFS_IMG_DIR_manipulation+'/*'))
gt_num_toal = len(glob.glob(IFS_IMG_MASK_DIR_manipulation+'/*'))

print('img_num_toal = ',img_num_toal)#一共1064幅
print('gt_num_toal = ',gt_num_toal)

Cov_imgS = []
Cov_labelS = []

for i, img in enumerate(glob.glob(IFS_IMG_DIR_manipulation+'/*')):

    sys.stdout.write('\ri: %d\t%s'%(i+1, os.path.basename(img)))
    sys.stdout.flush()
    
    image = plt.imread(img)
    print()
    print('image.shape[2]=',image.shape[2])
    if image.shape[2]==4:
        image = image[:,:,0:3]
        image = np.array(image)
    print('image.shape[2]=',image.shape[2])
    print()
    
    image_mani=scipy.misc.imresize(image,(256,256,3),interp='bilinear',mode=None)
    plt.imsave(fname='../data_256/IFS/img/'+os.path.basename(img),arr=image_mani)
    mask_mani = plt.imread('/home/shizenan/Datasets/IFS-TC-phase2-shi/img_mask/'+os.path.basename(img))
    if len(mask_mani.shape)==3:
        mask_mani = mask_mani[:,:,0]
    print('mask_mani',mask_mani.shape)
    if len(mask_mani.shape)==4:
        mask_mani = mask_mani[:,:,0]
    print('mask_mani',mask_mani.shape)
    
    
    mask_mani = scipy.misc.imresize(mask_mani,(256,256),interp='bilinear',mode=None)
    class_labels_tensor = np.float32(mask_mani==0)
    plt.imsave(fname='../data_256/IFS/GT/'+os.path.basename(img),arr=class_labels_tensor, cmap=plt.cm.gray)
    
    Cov_imgS.append(image_mani)
    Cov_labelS.append(class_labels_tensor)




np.save('../dataset_npy/IFS/IFS_imgS.npy', Cov_imgS)
np.save('../dataset_npy/IFS/IFS_labelS.npy',Cov_labelS)



img_num_toal =  441
gt_num_toal =  441
i: 1	279.png
image.shape[2]= 4
image.shape[2]= 3



/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


mask_mani (1536, 2048)
mask_mani (1536, 2048)
i: 2	103.png
image.shape[2]= 3
image.shape[2]= 3



/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


mask_mani (768, 1024)
mask_mani (768, 1024)
i: 3	98.png
image.shape[2]= 4
image.shape[2]= 3

mask_mani (1536, 2048)
mask_mani (1536, 2048)
i: 4	73.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 5	46.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (765, 1024)
mask_mani (765, 1024)
i: 6	294.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 7	69.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 8	396.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (1200, 1600)
mask_mani (1200, 1600)
i: 9	78.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 10	429.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (691, 922)
mask_mani (691, 922)
i: 11	80.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (765, 1024)
mask_mani (765, 1024)
i: 12	265.png
image.shape[2]= 4
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 13	336.png
ima

i: 89	112.png
image.shape[2]= 4
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 90	109.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 91	232.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 92	177.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 93	221.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 94	195.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 95	357.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 96	305.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (765, 1024)
mask_mani (765, 1024)
i: 97	426.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (1000, 1504)
mask_mani (1000, 1504)
i: 98	101.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 99	285.png
image.shape[2]= 4
image.shape[2]= 

i: 175	189.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 176	222.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 177	277.png
image.shape[2]= 4
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 178	126.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 179	91.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 180	334.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (771, 1024)
mask_mani (771, 1024)
i: 181	81.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 182	162.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 183	377.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (765, 1024)
mask_mani (765, 1024)
i: 184	29.png
image.shape[2]= 4
image.shape[2]= 3

mask_mani (600, 800)
mask_mani (600, 800)
i: 185	261.png
image.shape[2]= 3
image.shape[

mask_mani (2304, 3072)
mask_mani (2304, 3072)
i: 261	205.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 262	320.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (1944, 2592)
mask_mani (1944, 2592)
i: 263	37.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (2304, 3072)
mask_mani (2304, 3072)
i: 264	190.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 265	44.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 266	398.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 267	352.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (806, 1075)
mask_mani (806, 1075)
i: 268	240.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 269	169.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 270	361.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (3240, 4320)
mask_mani (32

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 346	119.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 347	373.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (1067, 1600)
mask_mani (1067, 1600)
i: 348	39.png
image.shape[2]= 4
image.shape[2]= 3

mask_mani (1536, 2048)
mask_mani (1536, 2048)
i: 349	433.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (1944, 2592)
mask_mani (1944, 2592)
i: 350	313.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (765, 1024)
mask_mani (765, 1024)
i: 351	435.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (806, 1075)
mask_mani (806, 1075)
i: 352	33.png
image.shape[2]= 4
image.shape[2]= 3

mask_mani (2758, 4150)
mask_mani (2758, 4150)
i: 353	123.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 354	1.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (1536, 2048)
mask_mani (1536, 2048)
i: 355	260.png
image.shape[2]= 4
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (7

i: 431	120.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 432	18.png
image.shape[2]= 4
image.shape[2]= 3

mask_mani (1200, 1800)
mask_mani (1200, 1800)
i: 433	286.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 434	258.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (1936, 2592)
mask_mani (1936, 2592)
i: 435	238.png
image.shape[2]= 4
image.shape[2]= 3

mask_mani (1536, 2048)
mask_mani (1536, 2048)
i: 436	62.jpg
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 437	124.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 438	50.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (768, 1024)
mask_mani (768, 1024)
i: 439	268.png
image.shape[2]= 3
image.shape[2]= 3

mask_mani (771, 1024)
mask_mani (771, 1024)
i: 440	15.png
image.shape[2]= 4
image.shape[2]= 3

mask_mani (933, 1600)
mask_mani (933, 1600)
i: 441	110.png
image.shape[2]= 3
image

## reample_feature_compute

In [14]:

Cov_img = np.load('../dataset_npy/IFS/IFS_imgS.npy')
Cov_img_feat = reample_feature_computer1(Cov_img)
np.save('../dataset_npy/IFS/IFS_img_feat.npy', Cov_img_feat)


np.shape(imgs)[0]= 441
0imgsize: (256, 256, 3)
1imgsize: (256, 256, 3)
2imgsize: (256, 256, 3)
3imgsize: (256, 256, 3)
4imgsize: (256, 256, 3)
5imgsize: (256, 256, 3)
6imgsize: (256, 256, 3)
7imgsize: (256, 256, 3)
8imgsize: (256, 256, 3)
9imgsize: (256, 256, 3)
10imgsize: (256, 256, 3)
11imgsize: (256, 256, 3)
12imgsize: (256, 256, 3)
13imgsize: (256, 256, 3)
14imgsize: (256, 256, 3)
15imgsize: (256, 256, 3)
16imgsize: (256, 256, 3)
17imgsize: (256, 256, 3)
18imgsize: (256, 256, 3)
19imgsize: (256, 256, 3)
20imgsize: (256, 256, 3)
21imgsize: (256, 256, 3)
22imgsize: (256, 256, 3)
23imgsize: (256, 256, 3)
24imgsize: (256, 256, 3)
25imgsize: (256, 256, 3)
26imgsize: (256, 256, 3)
27imgsize: (256, 256, 3)
28imgsize: (256, 256, 3)
29imgsize: (256, 256, 3)
30imgsize: (256, 256, 3)
31imgsize: (256, 256, 3)
32imgsize: (256, 256, 3)
33imgsize: (256, 256, 3)
34imgsize: (256, 256, 3)
35imgsize: (256, 256, 3)
36imgsize: (256, 256, 3)
37imgsize: (256, 256, 3)
38imgsize: (256, 256, 3)
39imgsize: (

319imgsize: (256, 256, 3)
320imgsize: (256, 256, 3)
321imgsize: (256, 256, 3)
322imgsize: (256, 256, 3)
323imgsize: (256, 256, 3)
324imgsize: (256, 256, 3)
325imgsize: (256, 256, 3)
326imgsize: (256, 256, 3)
327imgsize: (256, 256, 3)
328imgsize: (256, 256, 3)
329imgsize: (256, 256, 3)
330imgsize: (256, 256, 3)
331imgsize: (256, 256, 3)
332imgsize: (256, 256, 3)
333imgsize: (256, 256, 3)
334imgsize: (256, 256, 3)
335imgsize: (256, 256, 3)
336imgsize: (256, 256, 3)
337imgsize: (256, 256, 3)
338imgsize: (256, 256, 3)
339imgsize: (256, 256, 3)
340imgsize: (256, 256, 3)
341imgsize: (256, 256, 3)
342imgsize: (256, 256, 3)
343imgsize: (256, 256, 3)
344imgsize: (256, 256, 3)
345imgsize: (256, 256, 3)
346imgsize: (256, 256, 3)
347imgsize: (256, 256, 3)
348imgsize: (256, 256, 3)
349imgsize: (256, 256, 3)
350imgsize: (256, 256, 3)
351imgsize: (256, 256, 3)
352imgsize: (256, 256, 3)
353imgsize: (256, 256, 3)
354imgsize: (256, 256, 3)
355imgsize: (256, 256, 3)
356imgsize: (256, 256, 3)
357imgsize: 